In this notebook we will calculate the rate of in-hospital mortality across all facilities in New York State. Then we will look at a single hospital's mortality data by [APR DRG](http://solutions.3m.com/wps/portal/3M/en_US/Health-Information-Systems/HIS/Products-and-Services/Products-List-A-Z/APR-DRG-Software/). APR DRGs are a form of [diagnosis related group](https://en.wikipedia.org/wiki/Diagnosis-related_group) which are often tied to payment. They take into account both the diagnoses and procedures performed.

In [27]:
import pandas as pd

We leverage the server side to do the heavy lifting in computing the in-hospital mortality rates across the state.

In [28]:
base_url = "https://health.data.ny.gov/resource/rmwa-zns4"

In [29]:
pd.read_json(base_url + "?" + "$select=count(*)")

,count
0,2365208


In [30]:
url_count_by_disposition = base_url + "?" + "$select=count(*),patient_disposition&$group=patient_disposition"
print(url_count_by_disposition)
pd.read_json(url_count_by_disposition)

https://health.data.ny.gov/resource/rmwa-zns4?$select=count(*),patient_disposition&$group=patient_disposition


,count,patient_disposition
0,152,Medicaid Cert Nursing Facility
1,121,Critical Access Hospital
2,3789,Medicare Cert Long Term Care Hospital
3,2504,Cancer Center or Children's Hospital
4,1544,Hosp Basd Medicare Approved Swing Bed
5,725,Federal Health Care Facility
6,8441,Another Type Not Listed
7,3731,Court/Law Enforcement
8,10540,Hospice - Home
9,6268,Facility w/ Custodial/Supportive Care


In [31]:
url_facility_discharge_count = base_url + "?$select=count(*),facility_name&$group=facility_name"
print(url_facility_discharge_count)
facility_discharge_count_df = pd.read_json(url_facility_discharge_count)
facility_discharge_count_df.columns = ["discharge_count", "facility_name"]

https://health.data.ny.gov/resource/rmwa-zns4?$select=count(*),facility_name&$group=facility_name


In [32]:
facility_discharge_count_df.head(5)

,discharge_count,facility_name
0,86,Cuba Memorial Hospital Inc
1,1966,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...
2,5262,Olean General Hospital
3,2857,Brooks Memorial Hospital
4,1227,Womans Christian Assoc Hospital - WCA Hosp at ...


We pull separately the mortality data from SPARCS and then merge / join the data to the discharge count.

In [33]:
url_facility_expired_count = base_url + "?$select=count(*),facility_name&$group=facility_name&patient_disposition=Expired"
print(url_facility_expired_count)
facility_expired_count_df = pd.read_json(url_facility_expired_count)
facility_expired_count_df.columns = ["expired_count", "facility_name"]

https://health.data.ny.gov/resource/rmwa-zns4?$select=count(*),facility_name&$group=facility_name&patient_disposition=Expired


In [34]:
facility_expired_count_df.head(5)

,expired_count,facility_name
0,52,Memorial Hosp of Wm F & Gertrude F Jones A/K/A...
1,171,Olean General Hospital
2,73,Brooks Memorial Hospital
3,157,Woman's Christian Association
4,1,Westfield Memorial Hospital Inc


In [35]:
facility_discharged_with_expired_df = pd.merge(facility_discharge_count_df, facility_expired_count_df, on="facility_name")

In [36]:
facility_discharged_with_expired_df["expired_rate_per_discharge"] = facility_discharged_with_expired_df["expired_count"] / facility_discharged_with_expired_df["discharge_count"]

Mortality rates will vary by type of services that a hospital performs. For example a hospital with a large number of live births will have a lower mortality rate. This should be considered a first look of the data and more significant conclusions can be drawn when looking at the APR DRG level.

In [37]:
facility_discharged_with_expired_df.sort_values("expired_rate_per_discharge", ascending=False).head(25)

,discharge_count,facility_name,expired_count,expired_rate_per_discharge
125,2900,Calvary Hospital Inc,2391,0.824483
30,54,Monroe Community Hospital,7,0.129630
153,252,Henry J. Carter Specialty Hospital,17,0.067460
73,189,O'Connor Hospital,12,0.063492
4,18,Westfield Memorial Hospital Inc,1,0.055556
40,759,Soldiers and Sailors Memorial Hospital of Yate...,42,0.055336
143,10125,Mount Sinai Beth Israel Brooklyn,483,0.047704
38,556,Ira Davenport Memorial Hospital Inc,26,0.046763
184,6013,St. Joseph Hospital,275,0.045734
167,8884,St Johns Episcopal Hospital So Shore,403,0.045362


In [38]:
facility_discharged_with_expired_df.sort_values("expired_rate_per_discharge", ascending=False)[facility_discharged_with_expired_df["discharge_count"] >= 1000]

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,discharge_count,facility_name,expired_count,expired_rate_per_discharge
125,2900,Calvary Hospital Inc,2391,0.824483
143,10125,Mount Sinai Beth Israel Brooklyn,483,0.047704
184,6013,St. Joseph Hospital,275,0.045734
167,8884,St Johns Episcopal Hospital So Shore,403,0.045362
32,3642,Geneva General Hospital,165,0.045305
87,14150,Ellis Hospital,632,0.044664
18,1734,Degraff Memorial Hospital,70,0.040369
14,3957,Sisters of Charity Hospital - St Joseph Campus,159,0.040182
12,7713,Kenmore Mercy Hospital,303,0.039284
9,21186,Mercy Hospital of Buffalo,828,0.039082


In [39]:
hospital_url = base_url + "?facility_name=Coney%20Island%20Hospital&$limit=50000"
print(hospital_url)
hospital_df = pd.read_json(hospital_url)

https://health.data.ny.gov/resource/rmwa-zns4?facility_name=Coney%20Island%20Hospital&$limit=50000


In [43]:
# Uncomment the last line if there are API issues
#hospital_df.to_csv("./data/sparcs_hospital_apr_drg_2014.csv")
#hospital_df = pd.read_csv("./data/sparcs_hospital_apr_drg_2014.csv")

In [44]:
hospital_df.head(10)

,Unnamed: 0,abortion_edit_indicator,admit_day_of_week,age_group,apr_drg_code,apr_drg_description,apr_mdc_code,apr_mdc_description,apr_medical_surgical_description,apr_risk_of_mortality,...,operating_provider_license_number,patient_disposition,race,source_of_payment_1,source_of_payment_2,source_of_payment_3,total_charges,total_costs,type_of_admission,zip_code_3_digits
0,0,N,WED,30 to 49,753,BIPOLAR DISORDERS,19,Mental Diseases and Disorders,Medical,Minor,...,234646.0,Home or Self Care,Other Race,Medicaid,Medicaid,NaN,23450.98,17820.05,Emergency,112
1,1,N,WED,50 to 69,750,SCHIZOPHRENIA,19,Mental Diseases and Disorders,Medical,Minor,...,218003.0,Home or Self Care,Other Race,Medicare,Medicaid,NaN,30716.00,23340.63,Emergency,112
2,2,N,WED,30 to 49,140,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,4,Diseases and Disorders of the Respiratory System,Medical,Minor,...,235776.0,Home or Self Care,Black/African American,Medicare,Medicare,Medicaid,30174.63,22929.25,Emergency,112
3,3,N,WED,70 or Older,720,SEPTICEMIA & DISSEMINATED INFECTIONS,18,"Infectious and Parasitic Diseases, Systemic or...",Medical,Major,...,217963.0,Skilled Nursing Home,White,Medicare,Medicaid,NaN,30204.59,22952.01,Emergency,112
4,4,N,WED,70 or Older,140,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,4,Diseases and Disorders of the Respiratory System,Medical,Major,...,221582.0,Home or Self Care,Black/African American,Medicare,Medicare,Medicaid,26651.11,20251.78,Emergency,112
5,5,N,WED,50 to 69,190,ACUTE MYOCARDIAL INFARCTION,5,Diseases and Disorders of the Circulatory System,Medical,Major,...,199382.0,Short-term Hospital,White,Medicare,NaN,NaN,10611.60,8063.60,Emergency,112
6,6,N,WED,30 to 49,750,SCHIZOPHRENIA,19,Mental Diseases and Disorders,Medical,Minor,...,204304.0,Home or Self Care,Other Race,Medicaid,NaN,NaN,20958.05,15925.71,Emergency,112
7,7,N,WED,70 or Older,463,KIDNEY & URINARY TRACT INFECTIONS,11,Diseases and Disorders of the Kidney and Urina...,Medical,Moderate,...,241803.0,Home w/ Home Health Services,White,Medicare,Medicare,Medicaid,27150.89,20631.55,Emergency,112
8,8,N,WED,30 to 49,750,SCHIZOPHRENIA,19,Mental Diseases and Disorders,Medical,Minor,...,218003.0,Home or Self Care,White,Self-Pay,NaN,NaN,15168.03,11525.96,Emergency,112
9,9,N,WED,70 or Older,720,SEPTICEMIA & DISSEMINATED INFECTIONS,18,"Infectious and Parasitic Diseases, Systemic or...",Medical,Extreme,...,235776.0,Skilled Nursing Home,White,Medicare,Medicare,Private Health Insurance,30454.78,23142.13,Emergency,112


We create a new column in the dataframe which contains the DRG and the description.

In [45]:
hospital_df["apr_drg_with_code"] = hospital_df.apply(
    lambda x: str("00" + str(int(x["apr_drg_code"])))[-3:] + " - " + x["apr_drg_description"], axis=1)

For hospital stays longer than 120 days are coded as `"120 +"`. We convert these to 120 days so we can convert the values into a numeric values.

In [46]:
hospital_df["length_of_stay"] = hospital_df.apply(
    lambda x: 120 if "120 +" == x["length_of_stay"] else int(x["length_of_stay"]), axis=1)

In [47]:
hospital_df["length_of_stay"].count()

15436

We need to separately group the data for all discharges and for the mortality data.

In [48]:
import numpy as np
apr_drgs_df = hospital_df.groupby("apr_drg_with_code")["length_of_stay"].agg({"discharges": np.size, "total_days": np.sum, 
                                                                "mean_length_of_stay": np.mean})

In [49]:
apr_drgs_expired_df=hospital_df[hospital_df["patient_disposition"] == "Expired"].groupby("apr_drg_with_code")["length_of_stay"].agg({"number_of_in_hospital_deaths": np.size})

In [50]:
merged_apr_drgs_df = pd.merge(apr_drgs_df.reset_index(level=["apr_drg_with_code"]),apr_drgs_expired_df.reset_index(level=["apr_drg_with_code"]), on="apr_drg_with_code")

In [51]:
merged_apr_drgs_df.head(5)

,apr_drg_with_code,discharges,total_days,mean_length_of_stay,number_of_in_hospital_deaths
0,004 - TRACHEOSTOMY W MV 96+ HOURS W EXTENSIVE ...,9,524,58.222222,4
1,005 - TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSIV...,58,2901,50.017241,16
2,020 - CRANIOTOMY FOR TRAUMA,13,142,10.923077,2
3,021 - CRANIOTOMY EXCEPT FOR TRAUMA,27,624,23.111111,5
4,024 - EXTRACRANIAL VASCULAR PROCEDURES,6,17,2.833333,1


In [52]:
merged_apr_drgs_df.apr_drg_with_code.count()

90

In [53]:
merged_apr_drgs_df["number_hospital_deaths_per_discharge"] = merged_apr_drgs_df["number_of_in_hospital_deaths"] / merged_apr_drgs_df["discharges"]

In [54]:
merged_apr_drgs_df["number_hospital_deaths_per_patient_days"] = merged_apr_drgs_df["number_of_in_hospital_deaths"] / merged_apr_drgs_df["total_days"]

In [55]:
merged_apr_drgs_df.sort_values("number_hospital_deaths_per_discharge", ascending=False)[merged_apr_drgs_df["discharges"] >= 10]

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


,apr_drg_with_code,discharges,total_days,mean_length_of_stay,number_of_in_hospital_deaths,number_hospital_deaths_per_discharge,number_hospital_deaths_per_patient_days
24,196 - CARDIAC ARREST,13,37,2.846154,11,0.846154,0.297297
13,130 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATO...,41,907,22.121951,20,0.487805,0.022051
6,044 - INTRACRANIAL HEMORRHAGE,44,365,8.295455,14,0.318182,0.038356
45,281 - MALIGNANCY OF HEPATOBILIARY SYSTEM & PAN...,19,266,14.000000,6,0.315789,0.022556
74,720 - SEPTICEMIA & DISSEMINATED INFECTIONS,563,6977,12.392540,177,0.314387,0.025369
8,052 - NONTRAUMATIC STUPOR & COMA,28,188,6.714286,8,0.285714,0.042553
1,005 - TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSIV...,58,2901,50.017241,16,0.275862,0.005515
16,136 - RESPIRATORY MALIGNANCY,20,357,17.850000,4,0.200000,0.011204
3,021 - CRANIOTOMY EXCEPT FOR TRAUMA,27,624,23.111111,5,0.185185,0.008013
72,710 - INFECTIOUS & PARASITIC DISEASES INCLUDIN...,43,1114,25.906977,7,0.162791,0.006284


This analysis could be extended by computing the APR DRGs based mortality across one or more hospitals. That would allow an analyst to compare the rate at one facility agains